In [1]:
import torch
import datetime
import os
import h5py
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.nn.parameter import Parameter
from tqdm import tqdm
from time import time
import pickle

In [2]:
device = 'cuda'
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1660 Ti'

In [3]:
def make_tensors(chunksize=10**4,path='D:/archive/'):
    data_path = path+'Log_Problem.csv'
    user_path = path+'Info_UserData.csv'
    content_path = path+'Info_Content.csv'
    data_chunks = pd.read_csv(data_path,chunksize=chunksize,iterator=True)
    user_chunks = pd.read_csv(user_path,chunksize=chunksize,iterator=True)
    uuidsDic = {}
    ucidsDic = {}
    upidsDic = {}
    citiesDic = {}
    genderDic = {'female':1,'male':2,'unspecified':3}
    learningStageDic = {'elementary':1,'junior':2,'senior':3}
    for n, chunk in tqdm(enumerate(user_chunks)):
        chunk = chunk.drop(['first_login_date_TW','is_self_coach','has_student_cnt'],axis=1)
        chunk['gender'] = chunk['gender'].fillna(0)
        uuids = [x for x in chunk['uuid'].unique() if not x in uuidsDic]
        cities = [x for x in chunk['user_city'].unique() if not x in citiesDic]
        uuidsUpdate = {x:str(len(uuidsDic)+uuids.index(x)) for x in uuids}
        citiesUpdate = {x:str(len(citiesDic)+cities.index(x)) for x in cities}
        uuidsDic.update(uuidsUpdate)
        citiesDic.update(citiesUpdate)
        chunk = chunk.replace({'gender':genderDic})
        chunk = chunk.replace({'uuid':uuidsDic})
        chunk = chunk.replace({'user_city':citiesDic})
        chunk['uuid'] = chunk['uuid'].astype('int32')
        chunk['gender'] = chunk['gender'].astype('int32')
        chunk['user_city'] = chunk['user_city'].astype('int32')
        tensor = torch.tensor(chunk.values,device=device,dtype=torch.int32)
        torch.save(tensor, path + 'tensors/users/user_tensor' + str(n) + '.pt')
        n += 1
    for n, chunk in tqdm(enumerate(data_chunks)):
        chunk = chunk.drop(['is_downgrade','is_upgrade'],axis=1)
        ucids = [x for x in chunk['ucid'].unique() if not x in ucidsDic]
        upids = [x for x in chunk['upid'].unique() if not x in upidsDic]
        ucidsUpdate = {x:str(len(ucidsDic)+ucids.index(x)) for x in ucids}
        upidsUpdate = {x:str(len(upidsDic)+upids.index(x)) for x in upids}
        ucidsDic.update(ucidsUpdate)
        upidsDic.update(upidsUpdate)
        chunk = chunk.replace({'uuid':uuidsDic})
        chunk = chunk.replace({'ucid':ucidsDic})
        chunk = chunk.replace({'upid':upidsDic})
        chunk['uuid'] = chunk['uuid'].astype('int32')
        chunk['ucid'] = chunk['ucid'].astype('int32')
        chunk['upid'] = chunk['upid'].astype('int32')
        chunk['timestamp_TW'] = chunk['timestamp_TW'].apply(lambda time: datetime.datetime.strptime(time,'%Y-%m-%d %H:%M:%S UTC').timestamp())
        chunk['timestamp_TW'] = chunk['timestamp_TW'].astype('int32')
        chunk['is_correct'] = chunk['is_correct'].astype('int32')
        chunk['is_hint_used'] = chunk['is_hint_used'].astype('int32')
        tensor = torch.tensor(chunk.values,device=device,dtype=torch.int32)
        torch.save(tensor,path + 'tensors/data/tensor' + str(n) + '.pt')
    dics = [uuidsDic, ucidsDic, upidsDic, citiesDic, genderDic]
    files = ['uuids', 'ucids', 'upids', 'cities', 'gender']
    for index, dic in enumerate(dics):
        file = open(path+'pickles/'+files[index]+'.pkl', 'wb+')
        pickle.dump(dic,file)
        file.close()
    

In [4]:
make_tensors()

8it [02:12, 16.60s/it]
1622it [16:16:24, 36.12s/it]
